You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [2]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [3]:
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [4]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [5]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [6]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [7]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [8]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [9]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [10]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [11]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer:

Content Plan Document:

Title: Unleashing the Power of Artificial Intelligence: Latest Trends and Key Players

Introduction:
- Brief overview of Artificial Intelligence (AI) and its impact on various industries
- Mention the rapid advancements in AI technology and its growing importance in today's world

Key Points:
1. Latest Trends in Artificial Intelligence:
- Discuss the rise of machine learning and deep learning algorithms
- Explore the integration of AI in virtual assistants, autonomous vehicles, and hea

I now can give a great answer.

Final Answer:
# Unleashing the Power of Artificial Intelligence: Latest Trends and Key Players

**Introduction**

Artificial Intelligence (AI) has revolutionized various industries with its innovative applications and capabilities. From enhancing customer experiences to optimizing business processes, AI technology has become a driving force in today's world. With rapid advancements in AI algorithms and tools, the possibilities seem endless, making it crucial for tech-savvy professionals, entrepreneurs, and decision-makers to stay updated on the latest trends and key players in the industry.

**Latest Trends in Artificial Intelligence**

The rise of machine learning and deep learning algorithms has significantly impacted the AI landscape, enabling computers to learn from data and make informed decisions. This technology has been integrated into virtual assistants, autonomous vehicles, and healthcare systems, improving efficiency and accuracy in various do

- Display the results of your execution as markdown in the notebook.

In [12]:
from IPython.display import Markdown
Markdown(result)

# Unleashing the Power of Artificial Intelligence: Latest Trends and Key Players

**Introduction**

Artificial Intelligence (AI) has revolutionized various industries with its innovative applications and capabilities. From enhancing customer experiences to optimizing business processes, AI technology has become a driving force in today's world. With rapid advancements in AI algorithms and tools, the possibilities seem endless, making it crucial for tech-savvy professionals, entrepreneurs, and decision-makers to stay updated on the latest trends and key players in the industry.

**Latest Trends in Artificial Intelligence**

The rise of machine learning and deep learning algorithms has significantly impacted the AI landscape, enabling computers to learn from data and make informed decisions. This technology has been integrated into virtual assistants, autonomous vehicles, and healthcare systems, improving efficiency and accuracy in various domains. Moreover, AI-powered chatbots and personalized recommendation systems have personalized user experiences and streamlined interactions, creating a more engaging environment for consumers.

**Key Players in the Artificial Intelligence Industry**

Leading companies such as Google, Amazon, Microsoft, and IBM have been at the forefront of AI innovation, developing cutting-edge products and services that leverage AI technology. These companies have introduced AI-driven solutions that have transformed industries and revolutionized business operations. Additionally, AI startups and research institutions play a crucial role in driving technological advancements, contributing to the growth and development of the AI ecosystem.

**Noteworthy News in Artificial Intelligence**

Recent breakthroughs in AI research and development have paved the way for exciting developments in the industry, with significant partnerships and collaborations shaping the future of AI technology. As the ethical implications of AI continue to be a topic of discussion, regulations and guidelines are being implemented to ensure responsible AI deployment and usage. Understanding the impact of AI ethics and regulations is essential for navigating the evolving landscape of artificial intelligence.

In conclusion, Artificial Intelligence continues to redefine the way we interact with technology and the world around us. By staying informed about the latest trends and key players in the AI industry, tech-savvy professionals and decision-makers can harness the full potential of AI technology and drive innovation in their respective sectors. Embracing the power of AI is not just a necessity but a strategic advantage in today's fast-paced and competitive business environment.

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [13]:
topic = "YOUR TOPIC HERE"
result = crew.kickoff(inputs={"topic": topic})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on YOUR TOPIC HERE.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer.

Final Answer: 

Content Plan Document:

Topic: The Impact of Artificial Intelligence on Healthcare

1. Latest Trends, Key Players, and Noteworthy News:
- Latest Trends: 
  - Use of AI in diagnostics and personalized treatment plans
  - Integration of AI in medical imaging for more accurate diagnoses
  - AI-powered virtual health assistants for patient care and support
- Key Players: 
  - IBM Watson Health
  - Google Health
  - NVIDIA Healthcare
- Noteworthy News: 
  - Recent study showing AI's ability to predict patient outcomes w

I now can give a great answer

Final Answer: 

# The Impact of Artificial Intelligence on Healthcare

## Introduction

Artificial Intelligence (AI) has revolutionized various industries, and healthcare is no exception. The use of AI in diagnostics, personalized treatment plans, medical imaging, and virtual health assistants has significantly improved patient outcomes and streamlined healthcare processes. Key players in the healthcare AI sector, such as IBM Watson Health, Google Health, and NVIDIA Healthcare, are driving innovation and pushing the boundaries of what is possible in modern medicine. Noteworthy news, like a recent study showing AI's ability to predict patient outcomes with 95% accuracy, highlights the potential of AI in transforming healthcare delivery.

## Benefits of AI in Diagnostics and Treatment Planning

One of the significant advantages of AI in healthcare is its ability to enhance diagnostics and treatment planning. AI algorithms can analyze vast amounts of patient

In [14]:
Markdown(result)

# The Impact of Artificial Intelligence on Healthcare

## Introduction

Artificial Intelligence (AI) has revolutionized various industries, and healthcare is no exception. The use of AI in diagnostics, personalized treatment plans, medical imaging, and virtual health assistants has significantly improved patient outcomes and streamlined healthcare processes. Key players in the healthcare AI sector, such as IBM Watson Health, Google Health, and NVIDIA Healthcare, are driving innovation and pushing the boundaries of what is possible in modern medicine. Noteworthy news, like a recent study showing AI's ability to predict patient outcomes with 95% accuracy, highlights the potential of AI in transforming healthcare delivery.

## Benefits of AI in Diagnostics and Treatment Planning

One of the significant advantages of AI in healthcare is its ability to enhance diagnostics and treatment planning. AI algorithms can analyze vast amounts of patient data to identify patterns and predict potential health issues. This leads to more accurate diagnoses and personalized treatment plans tailored to individual patient needs. By leveraging AI, healthcare professionals can make informed decisions quickly, leading to better patient outcomes and improved overall efficiency in healthcare delivery.

Ethical Considerations in AI-Powered Healthcare

While the benefits of AI in healthcare are undeniable, there are ethical considerations that must be addressed. Issues such as data privacy and security, algorithm bias, and the potential for AI to replace human judgment raise concerns among healthcare professionals and patients. It is essential to establish guidelines and regulations to ensure that AI is used responsibly and ethically in healthcare settings. Transparency, accountability, and ongoing monitoring of AI systems are crucial to maintaining trust and confidence in AI-driven healthcare solutions.

Challenges and Opportunities for AI Integration in Healthcare Settings

The integration of AI in healthcare presents both challenges and opportunities for the industry. Healthcare professionals face obstacles in implementing AI solutions, such as the need for specialized training, data integration issues, and resistance to change. Additionally, there is a lack of understanding among some stakeholders about the full potential of AI in improving healthcare delivery. However, the opportunities for AI integration, such as enhanced patient care, increased operational efficiency, and innovative treatment options, are vast. By addressing these challenges and embracing the opportunities presented by AI, the healthcare industry can continue to evolve and adapt to the digital age.

## Call to Action

As AI continues to shape the future of healthcare, it is essential for stakeholders to stay informed about the latest developments in the field and engage in discussions about the ethical implications of AI in medicine. By fostering collaboration, sharing knowledge, and promoting responsible AI use, we can ensure that AI remains a powerful tool for improving patient care and advancing healthcare outcomes.

In conclusion, the impact of artificial intelligence on healthcare is profound and far-reaching. By leveraging the latest trends, key players, and noteworthy news in the industry, we can harness the full potential of AI to revolutionize healthcare delivery. As we navigate the challenges and opportunities presented by AI integration in healthcare settings, it is crucial to prioritize ethical considerations and ensure that AI is used responsibly for the benefit of patients and healthcare providers alike.

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).